We used crontask to poll http://api.citybik.es/v2/networks/citybikes-helsinki every hour from 2018-09-18 to 2018-10-10. We used bokeh library to visualize the data and pandas to handle csv files. Unfortunately, due to some setbacks we were unable to complete the asked model and route for the delivery truck. This could be achieved by using grouped_df variable. It holds sorted stations according to usage_ratio for each hour in a day.

In [1]:
import pandas as pd
import numpy as np

from os import listdir, getcwd
from os.path import isfile, join, dirname

path = getcwd() + '/data_hourly/'

all_files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]

df = pd.DataFrame()
df = pd.concat((pd.read_csv(f, encoding='utf8', engine='python', sep=';', index_col=0) for f in all_files))
df['total_slots'] = df.empty_slots + df.free_bikes
df['usage_ratio'] = df.empty_slots / df.total_slots
df['datetime'] = pd.DatetimeIndex(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['weekday'] = df['datetime'].apply(lambda x: x.weekday())
df = df.drop('extra', axis=1)

#Extract stations, min and max dates
stations = pd.DataFrame(df[:255], columns=['id', 'name', 'latitude', 'longitude'])
min_date = min(df['timestamp'])
max_date = max(df['timestamp'])


In [2]:
#Group by weekday, and hour of day, sort according to usage
grouped_df = df.groupby(['weekday', df.timestamp.dt.hour]).apply(pd.DataFrame.sort_values, 'usage_ratio', ascending=False)
#Example output
grouped_df.loc[(grouped_df.weekday == 0) & (grouped_df.timestamp.dt.hour == 0), ['name', 'usage_ratio']]

name  usage_ratio
weekday timestamp                                           
0       0         145             Esterinportti          1.0
                  131            Cygnauksenkatu          1.0
                  33                 Veturitori          1.0
                  236                Keilaranta          1.0
                  42                Elimäenkatu          1.0
                  137                 Annankatu          1.0
                  50                  Leiritori          1.0
                  157               Pernajantie          1.0
                  59        Hylkeenpyytäjänkatu          1.0
                  65               Piispansilta          1.0
                  76                  Linnuntie          1.0
                  81                  Itäportti          1.0
                  82                Tapionaukio          1.0
                  140               Gyldenintie          1.0
                  147        Rautatieläisenkatu          1.0
                  90                 Seurasaari          1.0
                  140               Gyldenintie          1.0
                  27      Sörnäisten metroasema          1.0
                  241               Maarinranta          1.0
                  122         Kaisaniemenpuisto          1.0
                  253           Korppaanmäentie          1.0
                  157               Pernajantie          1.0
                  108                 Radiokatu          1.0
                  109            Hertanmäenkatu          1.0
                  122         Kaisaniemenpuisto          1.0
                  90                 Seurasaari          1.0
                  236                Keilaranta          1.0
                  117               Designmuseo          1.0
                  241               Maarinranta          1.0
                  33                 Veturitori          1.0
...                                         ...          ...
                  219              Tilkanvierto          0.0
                  46        Juhana Herttuan tie          0.0
                  219              Tilkanvierto          0.0
                  128                     Baana          0.0
                  28             Heikkilänaukio          0.0
                  15            Liisanpuistikko          0.0
                  28             Heikkilänaukio          0.0
                  10   Munkkivuoren ostoskeskus          0.0
                  15            Liisanpuistikko          0.0
                  21                Apollonkatu          0.0
                  22                 Töölönkatu          0.0
                  21                Apollonkatu          0.0
                  15            Liisanpuistikko          0.0
                  22                 Töölönkatu          0.0
                  28             Heikkilänaukio          0.0
                  10   Munkkivuoren ostoskeskus          0.0
                  46        Juhana Herttuan tie          0.0
                  192          Länsisatamankatu          0.0
                  18                 Eläinmuseo          NaN
                  83                   Jäähalli          NaN
                  206   Eteläinen Hesperiankatu          NaN
                  18                 Eläinmuseo          NaN
                  83                   Jäähalli          NaN
                  206   Eteläinen Hesperiankatu          NaN
                  18                 Eläinmuseo          NaN
                  83                   Jäähalli          NaN
                  206   Eteläinen Hesperiankatu          NaN
                  18                 Eläinmuseo          NaN
                  83                   Jäähalli          NaN
                  206   Eteläinen Hesperiankatu          NaN

[1020 rows x 2 columns]